# 1.Introduction

This Notebook tutorial will allow you to perform Accessory authentication with TrustFLEX and Trust&GO devices.

## 1.1.Accessory authentication Transactions description

The CryptoAuthentication device supports different modes of operation with MAC command. The mode used here is “Move the Challenge to TempKey” where the host sends a random seed to the device and the device generates a nonce (number used once) which incorporates it. The generated nonce here is the challenge. The application sends the MAC command to the device to generate response with the challenge and the symmetric key in it. The generated response is sent to the application and the application performs the same operation done on the device side to generate the response. Finally, the application compares the response from the device and its own calculated response to authenticate. 

## 1.2.Tutorial Prerequisites:
    

The following code is runs on python 3.x environment. The folowing modules are also needed to run the example

1. cryptoauthlib
2. cryptography
3. ipywidgets

The next step of the tutorial with attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

In [ ]:
!python -m pip install -r ../../../requirements.txt

## 1.3. Load the necessary modules and helper functions

This Tutorial uses a number of python helper functions that are loaded by the following cell.
The below code does the following things:
1. Setup of cryptography module
2. Adds helper functions to get device name and device type

In [ ]:
from cryptoauthlib import *
from cryptoauthlib.iface import *
from common import *
from accessory_auth_helper import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import argparse,time
print("Importing modules - Successful")

# 2. Tutorial

This tutorial will guide you through the following steps:
1. Establish the communication between host (PC) and TrustFLEX
2. Generate MAC from device (TrustFLEX)
3. Verify Expected MAC on Host

## 2.1. Establish the communication between host (PC) and TrustFLEX
1. Initialize the library which also loads it
2. Check if the correct device is connected to the PC. If the wrong device is connected, a ValueError excpetion will be raised.

### 2.1.1. Select the CryptoAuthLib Hardware Abstraction layer to KIT USB HID


In [ ]:
ATCA_KIT_I2C_IFACE = 1
cfg = cfg_ateccx08a_kithid_default()
cfg.devtype = get_device_type_id('ATECC608A')
cfg.cfg.atcahid.dev_interface = ATCA_KIT_I2C_IFACE
cfg.cfg.atcahid.dev_identity = 0x6C #TFLXTLS device I2C address
ATCA_SUCCESS = 0
print("Set devtype as ECC608A - Successful")

### 2.1.2. Establish the connection with TrustFLEX device on the USB dongle and check its type
<font color=red>If the next Tutorial step yields an exception, the TrustFLEX is not correctly connected or of the wrong type.</font>

In [ ]:
# Initialize interface
assert atcab_init(cfg) == ATCA_SUCCESS, "Can't connect to the USB dongle"

# Get connected device type
info = bytearray(4)
assert atcab_info(info) == ATCA_SUCCESS, "Can't read the ECC608A device information"
dev_type = get_device_type_id(get_device_name(info))

# Checking if the connected device matches with selected device
if dev_type != cfg.devtype:
    assert atcab_release() == ATCA_SUCCESS
    raise ValueError('Device not supported')
    
print("Device initialization - Successful")

## 2.2. Generate MAC from TFLXTLS part
Generate the MAC with ECC608A device.The MAC is generated with the symmetric key in slot, Random number and with device serial number.The MAC generated from the device uses the Random number generator and the generated MAC is unique and will not repeat in future.

<center><img src="img/accessory_auth_image_source1.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
seed_in = bytearray(20)
rand_out = bytearray(32)
nonce = bytearray()
device_mac = bytearray(32)

# Generate the nonce in device and return the random number
assert atcab_nonce_rand(seed_in,rand_out) == ATCA_SUCCESS, "Random nonce from device failed"

# Calculate the nonce value on the host side
nonce.extend(rand_out[0:32])
nonce.extend(seed_in[0:20])
nonce.append(0x16)
nonce.append(0)
nonce.append(0)
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(bytes(nonce))
nonce = digest.finalize()

# Calculate the mac in device with its symmetric diversified key in slot
assert atcab_mac(MAC_MODE,SHARED_SECRET_SLOT,0,device_mac) == ATCA_SUCCESS, "MAC from Accessory device failed"
print("MAC Received from Accessory device:")
print(pretty_print_hex(device_mac, indent='    '))

## 2.3. Verify Expected MAC on Host
Calculate the MAC on the host side.The host with the help of the symmetric key, random nonce and with the TFLXTLS serial number generates the Expected MAC. It then compares the Expected MAC with the MAC sent from TFLXTLS device and executes application after successfull authentication.

<center><img src="img/accessory_auth_image_source2.png" alt="**Get your Secure Elements here!**" /></center>

In [ ]:
def mac_mac_resp_verify(b):
    serial_number = bytearray(9)
    host_mac = bytearray()
    Symmetric_key = bytearray()

    # Symmetric key to be used host mac calculation
    Symmetric_key = get_symmetric_key()

    #Read the serial number from device
    assert atcab_read_serial_number(serial_number) == ATCA_SUCCESS, "Serial number read from device failed"

    # Calculate the mac on the host side
    host_mac.extend(Symmetric_key[0:32])
    host_mac.extend(nonce[0:32])
    host_mac.append(8)
    host_mac.append(MAC_MODE)
    host_mac.append(SHARED_SECRET_SLOT)
    host_mac.append(0)
    host_mac.extend([0 for i in range(11)])
    host_mac.append(serial_number[8])
    host_mac.extend(serial_number[4:8])
    host_mac.extend(serial_number[0:2])
    host_mac.extend(serial_number[2:4])
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(bytes(host_mac))
    host_mac = digest.finalize()
    
    #uncomment following line to try wrong mac
    #host_mac = bytes(b'00000000000000000000000000000000')

    print("MAC calculated on host:")
    print(pretty_print_hex(host_mac, indent='    '))

    if (device_mac == host_mac):
        print('\nAccessory device authenticated successfully!')
        mac_verify.button_style = 'success'
    else:
        mac_verify.button_style = 'danger'
        print('\Accessory device not authenticated...')
        
mac_verify = widgets.Button(description = "Verify MAC", tooltip = 'MAC & MAC-Response Verify')
mac_verify.on_click(mac_mac_resp_verify)
display(mac_verify)